In [1]:
import cv2
import numpy as np
import imutils
import shutil
import sys
sys.path.append('../')
import data_prep
import pandas as pd
df = pd.read_csv('/home/payam/Dropbox/GRM/data/labeling.csv')

for row in df.iterrows():
    in_folder = '/home/payam/Dropbox/GRM/data/processed/'
    out_folder = '/home/payam/workspace/GRM/data/current/'
    name = str(int(row[1]['index'])-1).zfill(5)+'.tif'
    src = in_folder + name
    if row[1]['manual_label'] == 0:
        shutil.copy(src, out_folder+'negative/')
    else:
        shutil.copy(src, out_folder+'positive/')
        data_prep.augment(src, out_folder+'positive/')  

In [19]:
import os
from random import shuffle
# read files names 
pos = [ x for x in os.listdir(out_folder+'positive/')]
neg = [x for x in os.listdir(out_folder+'negative/')]

#create train and test set list
train_set = []
test_set = []
for f in pos:
    line = '{0}positive/{1} {2}'.format(out_folder, f, 1)
    if len(df)/2 < int(f.split('.')[0].split('_')[0]):
        train_set.append(line)
    else:
        # add just original image not augmented
        if not '_' in f:
            test_set.append(line)
for f in neg:
    line = '{0}negative/{1} {2}'.format(out_folder, f, 0)
    if len(df)/2 < int(f.split('.')[0]):
        train_set.append(line)
    else:
        test_set.append(line)
        
shuffle(train_set)
shuffle(test_set)

train_file = out_folder + 'train.txt'
test_file = out_folder + 'test.txt'

with open(train_file, 'w+') as trainFile:
    with open(test_file, 'w+') as testFile:
        for l in train_set:
            trainFile.write(l + '\n')
        for l in test_set:
            testFile.write(l + '\n')

In [22]:
import h5py
import os.path
from tflearn.data_utils import build_hdf5_image_dataset

# Build a HDF5 dataset (only required once)
build_hdf5_image_dataset(train_file, image_shape=(40, 40), mode='file', output_path=out_folder + 'train.h5',
                         categorical_labels=True, normalize=True)
build_hdf5_image_dataset(test_file, image_shape=(40, 40), mode='file', output_path=out_folder + 'test.h5',
                         categorical_labels=True, normalize=True)
# Load HDF5 dataset
_h5f = h5py.File(out_folder + 'train.h5', 'r')
X_train = _h5f['X']
Y_train = _h5f['Y']

h5f_ = h5py.File(out_folder + 'test.h5', 'r')
X_test = h5f_['X']
Y_test = h5f_['Y']

In [23]:
X_train.shape

(22893, 40, 40, 3)

In [6]:
train_file = out_folder + 'train.txt'
test_file = out_folder + 'test.txt'
with open(train_file, 'w+') as trainFile:
    with open(test_file, 'w+') as testFile:
        for p in pos:
            if p
            name = out_folder + 'positive/' + p

18602

In [16]:
img=cv2.imread('calendar.png')
cv2.imwrite('augmented/vertical.png',cv2.flip(img,1))
cv2.imwrite('augmented/horizontal.png',cv2.flip(img,0))
cv2.imwrite('augmented/flip90.png',imutils.rotate(img, 90))
cv2.imwrite('augmented/flip180.png',imutils.rotate(img, 180))
cv2.imwrite('augmented/flip270.png',imutils.rotate(img, 270))
cv2.imwrite('augmented/blured.png',cv2.blur(img,(5,5),0))
cv2.imwrite('augmented/gblured.png',cv2.GaussianBlur(img,(5,5),0))

True

In [6]:
'''
Parameters
----------
image : ndarray
    Input image data. Will be converted to float.
mode : str
    One of the following strings, selecting the type of noise to add:

    'gauss'     Gaussian-distributed additive noise.
    'poisson'   Poisson-distributed noise generated from the data.
    's&p'       Replaces random pixels with 0 or 1.
    'speckle'   Multiplicative noise using out = image + n*image,where
                n is uniform noise with specified mean & variance.
'''



import numpy as np
import os
import cv2
def noisy(noise_typ,image):
    if noise_typ == "gauss":
        row,col,ch= image.shape
        mean = 0
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = image + gauss
        return noisy
    elif noise_typ == "s&p":
        row,col,ch = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
        out[coords] = 1

        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
        out[coords] = 0
        return out
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy

In [13]:
cv2.imwrite('augmented/noisy.png', noisy('speckle', img))

True

In [19]:

df[df['manual_label'] != 0].shape

(1291, 13)

In [20]:
df.shape

(37204, 13)

In [21]:
df.head(5)

,index,building_index,lat0,lat1,lng0,lng1,manual_label,means,overlap,x0,x1,y0,y1
0,2.0,0.0,39.033528,39.033398,43.351591,43.351721,0.0,[ 255. 255. 255.],0.0,0.0,40.0,0.0,40.0
1,3.0,0.0,39.033528,39.033398,43.351656,43.351786,0.0,[ 255. 255. 255.],0.0,20.0,60.0,0.0,40.0
2,4.0,0.0,39.033528,39.033398,43.351721,43.351851,0.0,[ 255. 255. 255.],0.0,40.0,80.0,0.0,40.0
3,5.0,0.0,39.033528,39.033398,43.351786,43.351916,0.0,[ 255. 255. 255.],0.0,60.0,100.0,0.0,40.0
4,6.0,0.0,39.033528,39.033398,43.351851,43.351981,0.0,[ 255. 255. 255.],0.0,80.0,120.0,0.0,40.0


AttributeError: 'str' object has no attribute 'ends_with'